##INSTALL

In [1]:
!pip install google-cloud-vision;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##IMPORTS

In [2]:
import os
import sys
import time
from google.cloud import vision
import pandas as pd


In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "INSERT JSON FILE PATH PROVIDED BY GOOGLE - OCR KEY"
client = vision.ImageAnnotatorClient()

##FUNÇÃO | GOOGLE | OCR

PROJECT: XXX

OBJECT: IMAGE

ACTIVITY: OCR

ADDITIONAL: RegEX

In [9]:
import re
def clean_ocr_text(ocr):
    """ Esta função realiza a limpeza do texto utilizando expressões regulares (RegEX) e função replace.

    This function performs text cleaning using regular expressions (RegEX) and replace function. """
    
    del_ = ['Cabo', 'Caber', 'Cab', 'Cont', 'Est',
            'ey', 'Com', 'Proc', 'Cool', 'Ext',
            'Co', 'abo', 'OPR', 'Rst', 'Fst', 'nt', 
            'ANATEL', 'Cahn', 'oi','Caha', 'Caho',
            'Cant', 'ANATE', 'ANAT', 'Et', 'Came',
            'CABO ÓPTICO', 'V TAL', 'VTAL', '\n']     
            
    text = str(ocr)
    for item in del_:
        text = text.replace(str(item) , '').strip()
        text = text.replace(' ','')
        text = text.replace("&","8")
        text = text.replace("|","I")
        text = text.replace("×","X")
        text = text.replace("g","9")
        text = re.sub(r"[][().:;_\"'ـه₦-]","", text)

    return text

def extract_google_ocr(filepath):
    """ This function returns OCR (Optical character recognition) which is the process that
    converts text in image format to text in human-readable character format.
    
    Esta função retorna OCR (Optical character recognition) que é o processo que 
    converte um texto em formato de imagem em um texto em formato de caracteres legíveis."""

    with open(filepath, 'rb') as image_file:
        content = image_file.read()

    try:
        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        annotations = response.text_annotations
        
        texts = []
        for text in annotations:
            texts.append(text.description)
            
        return clean_ocr_text(texts[0])

    except:
        print('Google: fail to request OCR for:', filepath)

    return ''

In [10]:
img = '/content/drive/MyDrive/gestao_obras_2/Datasets/IMAGENS_TESTES/CROP_IMAGES/CROPPED/CROP_IMAGES/img_bot_114254201_10102022215403_predict.jpg'

In [11]:
extract_google_ocr(img)

'41K2P37F867PT04'

##Validacao

In [7]:
#Testes com a planilha em excel
def create_df(plan, campo):
    campo_gt = f'{str(campo)}_GT'
    campo_cellere = f'{str(campo)}_GOOGLE'
    result_campo = f'result_{str(campo)}'
    result_score_campo = f'result_score_{str(campo)}'
    campo_score = f'{str(campo)}_score'

    df = pd.read_excel(plan,'Batimento',engine='openpyxl')
    
    result = []
    for item in df[campo_gt]:
        score = apply_google_ocr(str(item))
        result.append(score)

    final_plan = df[[campo_gt, campo_cellere, result_campo, campo_score, result_score_campo]]
    final_plan['new_score'] = result
    final_plan['score_google'] = final_plan[campo_score]/0.96
    final_plan['final_score'] = final_plan['score_google']*final_plan['new_score']
    final_plan = final_plan.replace('\n', '')
    
    


    return final_plan


#### Lote: PASSAPORTE_PROD_NEO_passaporte_03_04_2022_14_09

In [8]:
path_plan = '/content/RESULT_ALL_OCR_plaquetas_31_10_22.xlsx' # lote 1
campo = 'GPON'
final_plan = create_df(path_plan , campo)
print(final_plan)
# print(len(final_plan.query(f"result_score_{campo} == 'SCORE_CORRETO' and {campo}_score > 0.5 and new_score < 0.5"))) # nº erros
# print(len(final_plan.query(f"result_score_{campo} == 'SCORE_CORRETO' and {campo}_score < 0.5 and new_score > 0.5"))) # nº erros
# print(len(final_plan.query(f"result_score_{campo} == 'SCORE_ERRADO' and {campo}_score > 0.5 and new_score < 0.5"))) # nº acertos
# print(len(final_plan.query(f"result_score_{campo} == 'SCORE_ERRADO' and {campo}_score < 0.5 and new_score > 0.5"))) # nº acertos

FileNotFoundError: ignored